In [29]:
import networkx
import numpy as np

from matplotlib import pyplot

import random

%matplotlib inline

In [107]:
grid_graph = networkx.grid_graph([5,5])

In [108]:
def generate_random_cz_schema(lattice_graph): # Hey turns out this method is just 
    loop_state = networkx.graphviews.generic_graph_view(lattice_graph)
    visited_edges = []
    cz_schema = {}
    layer_counter = 1
    while not networkx.is_empty(loop_state):
        layer_state = networkx.graphviews.generic_graph_view(loop_state)
        nodes_to_ignore = set()
        edges = []
        while not networkx.is_empty(layer_state):
            random_edge = random.choice(list(layer_state.edges))
            neighbours = [neighbour for neighbour in lattice_graph.adj[random_edge[0]]]
            neighbours += [neighbour for neighbour in lattice_graph.adj[random_edge[1]]]
            for n in neighbours:
                nodes_to_ignore.add(n)
            nodes_to_ignore.add(random_edge[0])
            nodes_to_ignore.add(random_edge[1])
            edges.append(random_edge)
            layer_state = networkx.graphviews.subgraph_view(
                loop_state,
                filter_node=networkx.filters.hide_nodes(nodes_to_ignore))
            assert(random_edge not in layer_state.edges)      
        for e in edges:
            visited_edges.append(e)
        cz_schema['layer_{}'.format(layer_counter)] = edges
        layer_counter += 1
        loop_state = networkx.graphviews.subgraph_view(
            lattice_graph,
            filter_edge=networkx.filters.hide_edges(visited_edges))
    return cz_schema


In [155]:
def index_to_coords(index, dimensions):
    def div_n_mod(val, div, mod):
        return (val//div)%mod
    divs = [1]
    for d in dimensions:
        divs.append(divs[-1]*d)
    return list(div_n_mod(index, div, mod) for div, mod in zip(divs, dimensions))
        
        

In [156]:
oop_state = networkx.graphviews.generic_graph_view(lattice_graph)
visited_edges = []
cz_schema = {}
layer_counter = 0
top_node = list(grid_graph.nodes)[-1]
dimension = len(top_node)
lengths = [dim+1 for dim in top_node]
visited_edges = []
while not networkx.is_empty(loop_state):
    layer_state = networkx.graphviews.generic_graph_viw(loop_state)
    excluded_nodes = set()
    main_axis = layer_counter % len(top_node)
    points = 1
    alt_axes = [i for i in range(dimension) if i != main_axis]
    alt_lengths = [lengths[i] for in in range(dimenson) if i != main_axis]
    for axis in alt_axes:
        points *= top_index(axis)
    for p in range(points):
        position = index_to_coords(p, alt_lengths)
        position.insert(0, maix_axis)
        for i in range(lengths[main_axis]):
            position[main_axis] = i
            #if position is in the layer_view, check for it's next neighbour
            #if so, add that edge, drop the nodes and their neighbours
        
        
        
    

SyntaxError: unexpected EOF while parsing (<ipython-input-156-a7b4ea28232f>, line 20)

In [170]:
a = [0,1,2,4,5]
a.insert(3,3)
a

[0, 1, 2, 3, 4, 5]

In [152]:
(59//(1))%3

2

In [167]:
grid_graph.adj

AtlasView({(0, 2): {}, (1, 1): {}, (2, 2): {}, (1, 3): {}})